In [1]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import requests
import torch
import pandas as pd
from tqdm import tqdm
import pandas as pd
import torch
from tqdm import tqdm
import time
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os
token_hf = os.environ['TOKEN']

/home/user/project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
far_right_EN = """Name: Mark Thompson
Age: 52
Sex: Male
Nationality: American
Political Group: Far-right conservative
Outlook: Distrusts mainstream media, believes "cancel culture" suppresses free speech. Views criticism of Right policies as unfair attacks. Likely to downplay offensiveness in right-wing tweets, but label progressive tweets as 'aggressive' or 'divisive.'"""

far_right_PL = """Imię i nazwisko: Mark Thompson
Wiek: 52
Płeć: Mężczyzna
Nacjonalność: Polak
Grupa polityczna: Skrajnie prawicowy konserwatysta
Poglądy: Nie ufa mediom głównego nurtu, uważa, że „kultura anulowania” tłumi wolność słowa. Postrzega krytykę polityki prawicowej jako niesprawiedliwe ataki. Prawdopodobnie bagatelizuje obraźliwe tweety prawicowe, ale określa tweety progresywne jako "agresywne" lub "dzielące"."""

far_right_RU = """Имя: Марк Томпсон
Возраст: 52
Пол: Мужчина
Национальность: Русский
Политическая группа: Крайне правый консерватор
Взгляды: Не доверяет основным СМИ, считает, что «культура отмены» подавляет свободу слова. Рассматривает критику правой политики как несправедливые нападки. Склонен преуменьшать оскорбительность правых твитов, но называть прогрессивные твиты „агрессивными“ или «вызывающими разногласия»."""

mod_cons_EN = """Name: Linda Carter
Age: 38
Sex: Female
Nationality: American
Political Group: Moderate conservative
Outlook: Values civility and tradition, dislikes extreme rhetoric from both sides. Might label overtly vulgar or conspiratorial tweets as offensive, even from the right."""

mod_cons_PL = """Imię i nazwisko: Linda Carter
Wiek: 38
Płeć: Kobieta
Nacjonalność: Polka
Grupa polityczna: Umiarkowany konserwatysta
Poglądy: Ceni uprzejmość i tradycję, nie lubi ekstremalnej retoryki z obu stron. Może uznać wulgarne lub konspiracyjne tweety za obraźliwe, nawet ze strony prawicy."""

mod_cons_RU = """Имя: Линда Картер
Возраст: 38
Пол: Женщина
Национальность: Русская
Политическая группа: Умеренно-консервативная
Взгляды: Ценит цивилизованность и традиции, не любит экстремальную риторику с обеих сторон. Может счесть оскорбительными откровенно вульгарные или конспирологические твиты, даже от правых."""

prog_left_EN = """Name: Aisha Patel
Age: 29
Sex: Female
Nationality: American
Political Group: Progressive left
Outlook: Highly sensitive to language targeting marginalized groups (racism, sexism). Likely to label tweets using terms like "socialist" pejoratively as offensive. Tolerant of aggressive progressive rhetoric if framed as social justice."""

prog_left_PL = """Imię i nazwisko: Aisha Patel
Wiek: 29 lat
Płeć: Kobieta
Nacjonalność: Polka
Grupa polityczna: Postępowa lewica
Perspektywy: Bardzo wrażliwa na język skierowany do grup marginalizowanych (rasizm, seksizm). Prawdopodobnie uzna za obraźliwe tweety używające pejoratywnych określeń takich jak „socjalista”. Toleruje agresywną, progresywną retorykę, jeśli jest oprawiona w ramy sprawiedliwości społecznej."""

prog_left_RU = """Имя: Айша Патель
Возраст: 29
Пол: Женщина
Национальность: Русская
Политическая группа: Прогрессивные левые
Взгляды: Очень чувствительна к языку, направленному на маргинализированные группы (расизм, сексизм). Скорее всего, сочтет оскорбительными твиты использующие уничижительные термины вроде «социалист». Терпимо относится к агрессивной прогрессивной риторике, если она оформлена как социальная справедливость."""

centrist_EN = """Name: James Cooper
Age: 45
Sex: Male
Nationality: American
Political Group: Centrist/Independent
Outlook: Pragmatic, dislikes partisan extremes. Harsher on tweets with misinformation or personal attacks, regardless of side. Less influenced by political loyalty; more likely to align annotations with factual accuracy and tone."""

centrist_PL = """Imię i nazwisko: James Cooper
Wiek: 45
Płeć: Mężczyzna
Nacjonalność: Polak
Grupa polityczna: Centrysta/Niezależny
Poglądy: Pragmatyczny, nie lubi skrajnych stronnictw. Ostrzejszy wobec tweetów zawierających dezinformacje lub ataki personalne, niezależnie od strony. Mniejszy wpływ lojalności politycznej; większe prawdopodobieństwo dostosowania adnotacji do faktycznej dokładności i tonu."""

centrist_RU = """Имя: Джеймс Купер
Возраст: 45
Пол: Мужчина
Национальность: Русский
Политическая группа: Центрист/Независимый
Взгляды: Прагматик, не любит крайностей в политике. Более жестко относится к твитам с дезинформацией или личными нападками, независимо от стороны. Менее подвержен влиянию политической лояльности; более склонен согласовывать аннотации с точностью фактов и тоном."""
system_prompt_EN = """Give the answer as only 0(not offensive) or 1(offensive). Do not include anything else in your response."""

user_text_EN = """You are a person of this personality: 
{personality}

How this person will label that statement (offensive or not):
```
{tweet}
```"""

system_prompt_PL = """Podaj odpowiedź jako 0 (nie obraźliwe) lub 1 (obraźliwe). Nie uwzględniaj niczego innego w swojej odpowiedzi."""

user_text_PL = """Jesteś osobą o tej osobowości:
{personality}

Jak ta osoba określi to stwierdzenie (obraźliwe lub nie): Jesteś osobą o tej osobowości:
```
{tweet}
```"""

system_prompt_RU = """Укажите в ответе только 0 (не оскорбительно) или 1 (оскорбительно). Не указывай в ответе ничего другого."""

user_text_RU = """Ты - человек этой личности:
{personality}

Как этот человек обозначит это высказывание (оскорбительное или нет):
```
{tweet}
```"""

# Define personalities
personalities = {
    "far_right_EN": far_right_EN,
    "far_right_PL": far_right_PL,
    "far_right_RU": far_right_RU,
    "mod_cons_EN": mod_cons_EN,
    "mod_cons_PL": mod_cons_PL,
    "mod_cons_RU": mod_cons_RU,
    "prog_left_EN": prog_left_EN,
    "prog_left_PL": prog_left_PL,
    "prog_left_RU": prog_left_RU,
    "centrist_EN": centrist_EN,
    "centrist_PL": centrist_PL,
    "centrist_RU": centrist_RU,
}

sys_prompt_mapping = {
    "system_prompt_EN": system_prompt_EN,
    "system_prompt_PL": system_prompt_PL,
    "system_prompt_RU": system_prompt_RU
}

user_prompt_mapping = {
    "user_text_EN": user_text_EN,
    "user_text_PL": user_text_PL,
    "user_text_RU": user_text_RU
}


In [3]:
model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto", token=token_hf
).eval()

processor = AutoProcessor.from_pretrained(model_id, token=token_hf)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.04s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [8]:
data = pd.read_parquet('data/Gemma/data.parquet')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 1 to 300
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Text_EN                    300 non-null    object 
 1   Text_PL                    300 non-null    object 
 2   Text_RU                    300 non-null    object 
 3   Translation_quality        300 non-null    float64
 4   Gemma_far_right_EN_answer  0 non-null      float64
 5   Gemma_far_right_PL_answer  0 non-null      float64
 6   Gemma_far_right_RU_answer  0 non-null      float64
 7   Gemma_mod_cons_EN_answer   0 non-null      float64
 8   Gemma_mod_cons_PL_answer   0 non-null      float64
 9   Gemma_mod_cons_RU_answer   0 non-null      float64
 10  Gemma_prog_left_EN_answer  0 non-null      float64
 11  Gemma_prog_left_PL_answer  0 non-null      float64
 12  Gemma_prog_left_RU_answer  0 non-null      float64
 13  Gemma_centrist_EN_answer   0 non-null      float64

In [9]:
# Function to get token probabilities from Gemma model in batches
def get_gemma_probabilities(system_prompts, user_prompts, processor, model):
    """
    Process multiple conversations in parallel with the Gemma model,
    returning probabilities for tokens "0" and "1".
    
    Args:
        system_prompts: List of system prompts
        user_prompts: List of user prompts
        processor: The processor/tokenizer for the model
        model: The Gemma model
        seed: Random seed for reproducibility
        
    Returns:
        A list of probability strings in format "prob_for_1, prob_for_0"
    """
    # Validate inputs
    if len(system_prompts) != len(user_prompts):
        raise ValueError("Number of system prompts must match number of user prompts")
        
    # Set random seed for reproducibility
    
    # Token IDs for Gemma (based on tokenizer results)
    token_id_0 = 236771  # Direct lookup token ID for "0"
    token_id_1 = 236770  # Direct lookup token ID for "1"
    
    # Prepare all conversations
    all_messages = [
        [
            {
                "role": "system",
                "content": [{"type": "text", "text": system_prompt}]
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt}
                ]
            }
        ]
        for system_prompt, user_prompt in zip(system_prompts, user_prompts)
    ]
    
    # Process each conversation
    probabilities = []
    
    for messages in all_messages:
        # Apply chat template
        inputs = processor.apply_chat_template(
            messages, 
            add_generation_prompt=True, 
            tokenize=True,
            return_dict=True, 
            return_tensors="pt"
        ).to(model.device, dtype=torch.bfloat16)
        
        # Forward pass through the model to get logits
        with torch.inference_mode():
            outputs = model(**inputs)
            logits = outputs.logits
            
            # Get probabilities for the next token (last position in sequence)
            next_token_logits = logits[0, -1, :]
            
            # Convert logits to probabilities using softmax
            probs = torch.nn.functional.softmax(next_token_logits, dim=-1)
            
            # Extract probabilities for "0" and "1" tokens
            prob_0 = probs[token_id_0].item()
            prob_1 = probs[token_id_1].item()
        
        # Format as requested: "prob_for_1, prob_for_0"
        probability_str = f"{prob_1:.4f}, {prob_0:.4f}"
        probabilities.append(probability_str)
    
    return probabilities



In [10]:
# Process data using the batch-enabled Gemma probability function
def process_data_with_gemma(data, personalities, user_prompt_mapping, sys_prompt_mapping, processor, model, batch_size=4):
    """
    Process data using the get_gemma_probabilities function in batches.
    
    Args:
        data: DataFrame containing the data
        personalities: Dictionary of personality prompts
        user_prompt_mapping: Dictionary mapping language codes to user prompt templates
        sys_prompt_mapping: Dictionary mapping language codes to system prompts
        processor: The processor/tokenizer for the model
        model: The Gemma model
        batch_size: Number of requests to batch together
    
    Returns:
        Updated DataFrame with Gemma token probabilities
    """
    # Ensure all columns exist with appropriate types
    for language_code in ['EN', 'PL', 'RU']:
        for personality_type in ["far_right", "mod_cons", "prog_left", "centrist"]:
            answer_col = f"Gemma_{personality_type}_{language_code}_answer"
            
            # Make sure answer column exists
            if answer_col not in data.columns:
                data[answer_col] = pd.NA
    
    # Create a list to track all items that need processing
    items_to_process = []
    
    # First scan to identify all items needing processing
    for i, row in data.iterrows():
        for language_code in ['EN', 'PL', 'RU']:
            for personality_type in ["far_right", "mod_cons", "prog_left", "centrist"]:
                personality_key = f"{personality_type}_{language_code}"
                answer_col = f"Gemma_{personality_type}_{language_code}_answer"
                
                # Only process if answer column is empty
                if pd.isna(data.loc[i, answer_col]):
                    tweet = row[f"Text_{language_code}"]
                    personality = personalities[personality_key]
                    prompt = user_prompt_mapping[f"user_text_{language_code}"].format(personality=personality, tweet=tweet)
                    
                    items_to_process.append({
                        'row_index': i,
                        'personality_type': personality_type,
                        'language_code': language_code,
                        'system_prompt': sys_prompt_mapping[f"system_prompt_{language_code}"],
                        'user_prompt': prompt
                    })
    
    print(f"Found {len(items_to_process)} items to process")
    
    # Process in batches
    for batch_start in tqdm(range(0, len(items_to_process), batch_size)):
        batch = items_to_process[batch_start:batch_start + batch_size]
        
        # Extract system prompts and user prompts for the batch
        system_prompts = [item['system_prompt'] for item in batch]
        user_prompts = [item['user_prompt'] for item in batch]
        
        # Get probabilities
        try:
            probability_strs = get_gemma_probabilities(
                system_prompts, 
                user_prompts,
                processor,
                model,
            )
            
            # Update dataframe with results
            for i, (item, prob_str) in enumerate(zip(batch, probability_strs)):
                row_idx = item['row_index']
                
                # Store the probability string in the answer column
                answer_col = f"Gemma_{item['personality_type']}_{item['language_code']}_answer"
                data.loc[row_idx, answer_col] = prob_str
        
        except Exception as e:
            print(f"Error processing batch starting at index {batch_start}: {e}")
        
        # Save progress periodically with error handling
        if batch_start % (batch_size * 5) == 0 or batch_start + batch_size >= len(items_to_process):
            try:
                # Create a copy of the dataframe with consistent types for saving
                save_df = data.copy()
                save_df.to_parquet("data/Gemma/data_gemma_progress.parquet")
                print(f"Saved progress after processing {batch_start + len(batch)} items")
            except Exception as e:
                print(f"Error saving data: {e}")
                # Try alternate save approach
                try:
                    # Save as CSV if parquet fails
                    data.to_csv("data/Gemma/data_gemma_progress_backup.tsv", sep="\t")
                    print("Saved backup as TSV instead")
                except:
                    print("Unable to save progress in any format. Continuing.")
            
        # Optional: Add a small delay between batches to prevent overloading
        time.sleep(0.1)
    
    return data

# Example usage:
"""
data = process_data_with_gemma(
    data, 
    personalities, 
    user_prompt_mapping, 
    sys_prompt_mapping,
    processor,
    model,
    batch_size=4  # Adjust based on GPU memory
)
"""

'\ndata = process_data_with_gemma(\n    data, \n    personalities, \n    user_prompt_mapping, \n    sys_prompt_mapping,\n    processor,\n    model,\n    batch_size=4  # Adjust based on GPU memory\n)\n'

In [11]:
results = process_data_with_gemma(
    data, 
    personalities, 
    user_prompt_mapping, 
    sys_prompt_mapping,
    processor,
    model,
    batch_size=180  # Adjust based on GPU memory
)

try:
    # Create a copy of the dataframe with consistent types for saving
    save_df = results.copy()
    save_df.to_parquet("data/Gemma/data_qwen_complete.parquet")
except Exception as e:
    print(f"Error saving data: {e}")
    # Try alternate save approach
    try:
        # Save as CSV if parquet fails
        results.to_csv("data/Gemma/data_qwen_complete_backup.tsv", sep='\t')
        print("Saved backup as TSV instead")
    except:
        print("Unable to save complete in any format. Continuing.")

Found 3600 items to process


  0%|          | 0/20 [00:00<?, ?it/s]

/tmp/ipykernel_14935/3438802214.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1152, 0.8848' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc[row_idx, answer_col] = prob_str
/tmp/ipykernel_14935/3438802214.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2172, 0.7828' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc[row_idx, answer_col] = prob_str
/tmp/ipykernel_14935/3438802214.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0450, 0.9550' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc[row_idx, answer_col] = prob_str
/tmp/ipykernel_14935/3438802214.py:76: FutureWarning

Saved progress after processing 180 items


 30%|███       | 6/20 [01:23<03:17, 14.08s/it]

...

Saved progress after processing 1980 items


 80%|████████  | 16/20 [03:42<00:55, 13.99s/it]

Saved progress after processing 2880 items


100%|██████████| 20/20 [04:37<00:00, 13.87s/it]

Saved progress after processing 3600 items


In [12]:
# Step 1: Split into two new DataFrames: P1 (prob. of 1) and P0 (prob. of 0)
P1 = results.iloc[:, 4:].applymap(lambda x: float(x.split(',')[0].strip()))
P0 = results.iloc[:, 4:].applymap(lambda x: float(x.split(',')[1].strip()))

# Step 2: Analyze each column
analysis = pd.DataFrame(index=results.iloc[:, 4:].columns)
analysis['Mean P(1)'] = P1.mean()
analysis['Mean P(0)'] = P0.mean()
analysis['Min P(1)'] = P1.min()
analysis['Max P(1)'] = P1.max()
analysis['Std P(1)'] = P1.std()
analysis['Sanity Check (P1 + P0 ≈ 1)'] = (P1 + P0).apply(lambda col: col.sub(1).abs().max() < 1e-6)

print(analysis)

                           Mean P(1)  Mean P(0)  Min P(1)  Max P(1)  Std P(1)  \
Gemma_far_right_EN_answer   0.646223   0.353777    0.0001    1.0000  0.444108   
Gemma_far_right_PL_answer   0.914615   0.085384    0.0016    1.0000  0.251575   
Gemma_far_right_RU_answer   0.694761   0.305236    0.0006    0.9999  0.429723   
Gemma_mod_cons_EN_answer    0.764271   0.235728    0.0002    1.0000  0.407719   
Gemma_mod_cons_PL_answer    0.795078   0.204921    0.0004    1.0000  0.383204   
Gemma_mod_cons_RU_answer    0.598543   0.401443    0.0002    1.0000  0.473233   
Gemma_prog_left_EN_answer   0.781958   0.218041    0.0001    1.0000  0.394417   
Gemma_prog_left_PL_answer   0.839429   0.160571    0.0002    1.0000  0.348964   
Gemma_prog_left_RU_answer   0.876710   0.123276    0.0005    1.0000  0.306586   
Gemma_centrist_EN_answer    0.626481   0.373518    0.0001    1.0000  0.469225   
Gemma_centrist_PL_answer    0.647344   0.352655    0.0002    1.0000  0.465694   
Gemma_centrist_RU_answer    

/tmp/ipykernel_14935/3857150553.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  P1 = results.iloc[:, 4:].applymap(lambda x: float(x.split(',')[0].strip()))
/tmp/ipykernel_14935/3857150553.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  P0 = results.iloc[:, 4:].applymap(lambda x: float(x.split(',')[1].strip()))
